## Засоби підготовки та аналізу даних
#### Шафранський Даніїл, ФБ-22. Лабораторна робота №2

Imports

In [1]:
import pandas as pd

In [2]:
import urllib.request
from datetime import datetime, date
import os
import csv

Variables

In [3]:
url_base=('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={''}&year1=1981&year2=2024&type=Mean')
dir = 'CSV_Files'
indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
if not os.path.exists(dir):
    os.makedirs(dir)

Download csv

In [4]:
def construct_url(province_id):
    return url_base.format(province_id)

def file_exists(directory, file_prefix):
    for file_name in os.listdir(directory):
        if file_name.startswith(file_prefix):
            return True
    return False

def download_file(url, file_path):
    with urllib.request.urlopen(url) as response:
        text = response.read()
        with open(file_path, 'wb') as file:
            file.write(text)

def update_file_if_needed(url, file_path):
    with urllib.request.urlopen(url) as response:
        new_text = response.read()
        with open(file_path, 'rb') as file:
            existing_text = file.read()
        if existing_text != new_text:
            with open(file_path, 'wb') as file:
                file.write(new_text)
            print('File updated:', file_path)
        else:
            print('File already exists and was not downloaded:', file_path)

def main():
    for province_id, index in indexes.items():
        file_prefix = 'NOAA_' + str(index) + '_'
        file_already_exists = file_exists(dir, file_prefix)

        if not file_already_exists:
            url = construct_url(province_id)
            date_and_time_time = datetime.now().strftime("%Y%m%d%H%M%S")
            file_name = file_prefix + date_and_time_time + '.csv'
            file_path = os.path.join(dir, file_name)
            download_file(url, file_path)
            print('File created:', file_name)
        else:
            file_path = next((os.path.join(dir, file_name) for file_name in os.listdir(dir) if file_name.startswith(file_prefix)), None)
            if file_path:
                update_file_if_needed(construct_url(province_id), file_path)

    print('Success')

main()

File already exists and was not downloaded: CSV_Files\NOAA_22_20240317005336.csv
File already exists and was not downloaded: CSV_Files\NOAA_24_20240317005337.csv
File already exists and was not downloaded: CSV_Files\NOAA_23_20240317005338.csv
File already exists and was not downloaded: CSV_Files\NOAA_25_20240317005338.csv
File already exists and was not downloaded: CSV_Files\NOAA_3_20240317005339.csv
File already exists and was not downloaded: CSV_Files\NOAA_4_20240317005340.csv
File already exists and was not downloaded: CSV_Files\NOAA_8_20240317005341.csv
File already exists and was not downloaded: CSV_Files\NOAA_19_20240317005342.csv
File already exists and was not downloaded: CSV_Files\NOAA_20_20240317005343.csv
File already exists and was not downloaded: CSV_Files\NOAA_21_20240317005344.csv
File already exists and was not downloaded: CSV_Files\NOAA_9_20240317005345.csv
File already exists and was not downloaded: CSV_Files\NOAA_10_20240317005346.csv
File already exists and was not 

Create dataframe

In [8]:
data_dir = "CSV_Files"
output_path = "full.csv"

column_names = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "Area"]
combined_data = pd.DataFrame(columns=column_names)

filenames = os.listdir(data_dir)

for filename in filenames:
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(data_dir, filename)

    df = pd.read_csv(file_path, skiprows=2, names=column_names)
    df["Year"] = df["Year"].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')

    region_id = int(filename.split('_')[1])
    df["Area"] = region_id

    df = df.drop(df.loc[df['VHI'] == -1].index).dropna()

    combined_data = pd.concat([combined_data, df], ignore_index=True)

combined_data.to_csv(output_path, index=False)

print(combined_data)

       Year  Week    SMN     SMT    VCI    TCI    VHI Area
0      1982   1.0  0.045  261.12  36.68  41.79  39.23   10
1      1982   2.0  0.041  262.61  36.46  35.10  35.78   10
2      1982   3.0  0.041  263.82  36.49  30.79  33.64   10
3      1982   4.0  0.038  264.71  33.46  28.66  31.06   10
4      1982   5.0  0.034  264.76  29.66  30.50  30.08   10
...     ...   ...    ...     ...    ...    ...    ...  ...
53595  2024   6.0  0.087  265.01  55.93  37.82  46.87    9
53596  2024   7.0  0.104  268.59  60.95  29.91  45.40    9
53597  2024   8.0  0.121  271.86  64.85  23.64  44.21    9
53598  2024   9.0  0.138  274.65  68.03  20.12  44.05    9
53599  2024  10.0  0.150  276.27  67.09  23.10  45.07    9

[53600 rows x 8 columns]


Read csv into dataframe

In [6]:
df=pd.read_csv('full.csv')

Ряд VHI для області за вказаний рік, пошук екстремумів (min та max)

In [7]:
def proc_1(dataframe, index, year):
    vhi = dataframe[(dataframe["Area"] == index) & (dataframe["Year"] == year)]['VHI']
    vhi_max = vhi.max()
    vhi_min = vhi.min()
    vhi_row = ', '.join([y.split('    ')[-1] for y in [x for x in str(vhi).split('\n')[:-1]]])
    return f"Ряд VHI для області за вказаний рік: {vhi_row}.\nМінімальний показник VHI для області з індексом {index} у {year} році складав {vhi_min}. Максимальний показник VHI для області з індексом {index} у {year} році складав {vhi_max}."

index = 1
year = 1991

print(proc_1(df, index, year))

Ряд VHI для області за вказаний рік: 33.17, 36.01, 38.33, 39.89, 38.53, 36.72, 35.60, 35.59, 36.77, 35.44, 31.98, 31.10, 29.06, 29.11, 34.67, 39.71, 46.18, 50.21, 52.39, 54.81, 56.40, 57.68, 59.95, 63.62, 67.10, 68.82, 71.57, 73.29, 72.34, 71.94, 71.29, 70.75, 69.76, 68.99, 65.85, 62.26, 60.28, 56.41, 52.48, 50.49, 48.03, 49.48, 52.52, 47.84, 42.39, 38.64, 38.21, 38.69, 36.86, 36.32, 34.72, 31.50.
Мінімальний показник VHI для області з індексом 1 у 1991 році складав 29.06. Максимальний показник VHI для області з індексом 1 у 1991 році складав 73.29.


Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25)

In [9]:
def proc_2_1(dataframe, years_start, years_end, indexes):
    years = range(years_start, years_end)
    vhi_by_index = {}
    for index in indexes:
        filtered_df = dataframe[(dataframe["Area"] == index) & (dataframe["Year"].isin(years))]
        vhi_by_index[index] = filtered_df["VHI"].tolist()

    for index, vhi in vhi_by_index.items():
        print(f"Ряд VHI для області з індексом {index} за вказаний діапазон років: {', '.join([str(x) for x in vhi])}")

starting_year = 1991
ending_year = 1995
province_indexes = [1, 3, 5]
proc_2_1(df, starting_year, ending_year, province_indexes)

def proc_2_2(dataframe, percent):
    yearly_affected_areas = dataframe[dataframe["VHI"] < 15].groupby("Year")["Area"].nunique()
    yearly_percentage_affected = yearly_affected_areas / df["Area"].unique().size
    years_with_extreme_drought = yearly_percentage_affected[yearly_percentage_affected >= percent].index.to_list()

    print("Роки, протягом яких екстримальна посуха торкнулися більше вказаного відсотка областей:")
    print(*years_with_extreme_drought, sep=", ")

proc_2_2(df, 0.1)

Ряд VHI для області з індексом 1 за вказаний діапазон років: 33.17, 36.01, 38.33, 39.89, 38.53, 36.72, 35.6, 35.59, 36.77, 35.44, 31.98, 31.1, 29.06, 29.11, 34.67, 39.71, 46.18, 50.21, 52.39, 54.81, 56.4, 57.68, 59.95, 63.62, 67.1, 68.82, 71.57, 73.29, 72.34, 71.94, 71.29, 70.75, 69.76, 68.99, 65.85, 62.26, 60.28, 56.41, 52.48, 50.49, 48.03, 49.48, 52.52, 47.84, 42.39, 38.64, 38.21, 38.69, 36.86, 36.32, 34.72, 31.5, 29.68, 29.92, 32.14, 33.45, 35.01, 36.33, 36.02, 35.83, 36.62, 36.33, 35.17, 36.95, 38.59, 40.89, 45.89, 48.55, 48.15, 49.09, 49.63, 49.02, 47.79, 47.25, 47.82, 50.19, 52.09, 52.89, 54.14, 53.77, 50.25, 43.65, 35.91, 29.69, 24.88, 22.33, 23.49, 26.9, 30.22, 32.59, 35.85, 42.02, 43.15, 42.82, 42.93, 40.03, 37.75, 37.53, 38.66, 41.08, 40.66, 39.96, 39.62, 38.93, 37.19, 37.02, 36.61, 36.55, 37.18, 38.57, 38.76, 39.94, 42.15, 44.13, 45.43, 47.96, 48.55, 49.79, 52.66, 53.81, 50.5, 49.18, 51.48, 53.99, 56.0, 57.09, 58.59, 62.56, 65.56, 66.56, 69.46, 72.1, 72.61, 70.13, 69.5, 70.3

Аналогічно для помірних посух

In [11]:
def proc_3(dataframe, percent):
    yearly_affected_areas = dataframe[(dataframe["VHI"] < 35) & (dataframe["VHI"] > 15)].groupby("Year")["Area"].nunique()
    yearly_percentage_affected = yearly_affected_areas / df["Area"].unique().size
    years_with_moderate_drought = yearly_percentage_affected[yearly_percentage_affected >= percent].index.to_list()

    print("Роки, протягом яких помірна посуха торкнулися більше вказаного відсотка областей:")
    print(*years_with_moderate_drought, sep=", ")

proc_3(df, 0.9)

Роки, протягом яких помірна посуха торкнулися більше вказаного відсотка областей:
1982, 1983, 1984, 1985, 1986, 1989, 1990, 1992, 1993, 1995, 1996, 2000, 2015, 2019
